### Engine DB Queries for Wash Case Reports

In [1]:
import os; 
import pandas as pd; pd.set_option('mode.chained_assignment',None) 
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import re
import xlrd
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
import time

from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, totalUP, wtf
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### 1) dates & accounts of interest

In [2]:
alerting_1 = datetime.date(2019,4,1)
alerting_2 = datetime.date(2019,8,31)

In [4]:
# query = """ 
#         SELECT  *
#         from order_fill_event 
#         where created between '{}' and '{}' and account_id = {};
#         """.format(alerting_1.strftime("%Y-%m-%d"),alerting_2.strftime("%Y-%m-%d"),950942)

# df = execute_query_data_frame(query,'engine'); df
# # output1a = df.groupby([df['created'].dt.month, 'pair']).sum().astype(int); output1a

In [ ]:
print(query)

#### 1A)  traded value($M) by Account (approximation- DO NOT USE)

In [ ]:
query = """ 
        SELECT  created::date, account_id as pair, sum(price*quantity)/1000000 as TVmil, count(distinct event_id) as count
        from order_fill_event 
        where created between '{}' and '{}' and account_id in {}
        group by account_id, created::date
        order by created::date, account_id;
        """.format(alerting_1.strftime("%Y-%m-%d"),alerting_2.strftime("%Y-%m-%d"),str(tuple([worst])))

df = execute_query_data_frame(query,'engine')
output1a = df.groupby([df['created'].dt.month, 'pair']).sum().astype(int); output1a

#### 2) traded value($M) TOTAL EXCHANGE

In [ ]:
query2 = """ 
        SELECT sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and side = 'buy') as firsttable 
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time 
        
        
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"))

output2 = execute_query_data_frame(query2,'engine'); output2

In [ ]:
query2 = """ 
        SELECT account_id, side, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in {x}) as firsttable 
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time 
        where account_id in {x}
        group by account_id, side
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"),x = str(tuple(remainingcpty)))

output2 = execute_query_data_frame(query2,'engine')
output2b = round(pd.pivot_table(output2,columns='side',index='account_id',values='valm'),2)

In [ ]:
output2b = round(pd.pivot_table(output2,columns='side',index='account_id',values='valm'),1)
output2b['TOTAL'] = output2b.sum(axis=1)
output2b['pct'] = round(output2b['TOTAL']/2325*100,0).astype(int).astype(str)+'%'
output2b.sort_values('TOTAL',ascending=False)

In [ ]:
outputnew = output2b.sort_values('TOTAL',ascending=False)
nonzero = outputnew.loc[outputnew['pct'] != '0%']
zero = pd.DataFrame(outputnew.loc[outputnew['pct'] == '0%',['buy','sell','TOTAL']].sum(axis=0)).transpose()

In [ ]:
outputnew2 = pd.concat([nonzero,zero],axis=0,sort=False).reset_index()
outputnew2.loc[outputnew2['index'] == 0,'pct'] = "<0%"

In [ ]:
outputnew2.to_csv('~/Desktop/no.csv')

#### 3) traded value($M) by Account & Side

In [ ]:
query3 = """ 
        SELECT account_id, trading_pair, firsttable.side, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in {x}) as firsttable
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time

        group by firsttable.side, account_id, trading_pair;
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"), x=str(tuple(worst)))

output3_D = execute_query_data_frame(query3,'engine')

In [ ]:
output3 = pd.pivot_table(output3_D, values='valm', index=['account_id','trading_pair'], columns=['side'], aggfunc=np.sum)
output3['TOTAL'] = output3.sum(axis=1)
round(output3,1)

In [ ]:
output3.reset_index().groupby('account_id').sum()

#### 4) traded value($M) by Account & Side

In [ ]:
query4 = """ 
        SELECT account_id, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity/1000000) as valM
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('hour', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in {x}) as firsttable
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1hr 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time
        
        where account_id = {y} and event_id in 
            (SELECT distinct event_id from order_fill_event
            where created between '{t1}' and '{t2}' and account_id = {z}
            group by event_id) 
        
        group by account_id;
        """

output4 = pd.DataFrame()

for i in pairlist:
    temp = execute_query_data_frame(query4.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"), x=str(tuple(worst)), y = i[0], z = i[1]),'engine')
    temp['pair'] = str(i)
    output4 = output4.append(temp, sort=True)
    
output4